<a href="https://colab.research.google.com/github/AleAguirreM/Analitica3AA/blob/main/b_preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import plotly.graph_objs as go
import plotly.express as px
import sys
from mlxtend.preprocessing import TransactionEncoder

In [ ]:
#conectamos drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
sys.path.append('/content/drive/MyDrive')

In [ ]:
import a_funciones as fn

In [ ]:
conn = sql.connect("/content/drive/MyDrive/db_movies3")

In [ ]:
curr = conn.cursor()

In [ ]:
curr.execute("SELECT name FROM sqlite_master WHERE type = 'table'")

In [ ]:
curr.fetchall()

[]

In [ ]:
pd.read_sql("select * from ratings", conn)

DatabaseError: Execution failed on sql 'select * from ratings': no such table: ratings

In [ ]:
#try:
    # Intentar agregar la columna fecha_fecha si no existe
    #curr.execute("ALTER TABLE ratings ADD COLUMN fecha_fecha DATE")
#except sql.OperationalError as e:
    # Si la columna ya existe, mostrar un mensaje de advertencia
    #print("La columna 'fecha_fecha' ya existe en la tabla 'ratings'.")

# Actualizar la nueva columna con los valores convertidos de la columna existente
#curr.execute("UPDATE ratings SET fecha_fecha = DATE(timestamp)")


In [ ]:
movies = pd.read_sql("select * from movies", conn)

In [ ]:
ratings = pd.read_sql("select * from ratings", conn)

In [ ]:

#####Exploración inicial #####

#Identificar campos de cruce y verificar que estén en mismo formato ####
### verificar duplicados

ratings.info()
ratings.head()
ratings.duplicated().sum()

movies.info()
movies.head()
movies.duplicated().sum()

ratings.rating.unique()


In [ ]:
#¿Cuántos usuarios han calificado al menos 50 películas diferentes?
consulta_sql = """
    SELECT userId, COUNT(DISTINCT movieId) AS numero_peliculas
    FROM ratings
    GROUP BY userId
    HAVING numero_peliculas >= 50
"""
pd.read_sql(consulta_sql, conn)

In [ ]:
###calcular la distribución de calificaciones
cr=pd.read_sql(""" select
                          "rating" as rating,
                          count(*) as conteo
                          from ratings
                          group by "rating"
                          order by conteo desc""", conn)
cr

In [ ]:
data  = go.Bar( x=cr.rating,y=cr.conteo, text=cr.conteo, textposition="outside")
Layout=go.Layout(title="Count of ratings",xaxis={'title':'Rating'},yaxis={'title':'Count'})
go.Figure(data,Layout)

Vemos que la pelicula es calificada generalmente cuando es buena o aceptable, la mayor concentracion de calficaciones se encuentra entre 3-5, la distribucion entre califaficaciones de 2.5 a 0.5, es realmente baja.

In [ ]:
### calcular cada usuario cuátos peliculas calificó
rating_users=pd.read_sql(''' select "userId" as user_id,
                         count(*) as cnt_rat
                         from ratings
                         group by "userId"
                         order by cnt_rat asc
                         ''',conn )
rating_users

In [ ]:
fig  = px.histogram(rating_users, x= 'cnt_rat', title= 'Hist frecuencia de numero de calificaciones por usuario')
fig.show()

En el gràfico vemos que es poco frecuente que los usarios califiquen menos de 500 peliculas, por lo que se acota los datos para ver una mejor distribuciòn. Hay datos atipicos, ya que se muestran usuarios que han calificado más de 2000 peliculas

In [ ]:
rating_users.describe()

En el describe se evidencia que el 75% de los usuarios ha calificado menos de 168 libros, que el minimo de calificaciones es de 20, que la mediana es 165 calificaciones aproximadamente y el total de usuarios es de 610.

In [ ]:
### excluimos los usuarios que tengan menos de 170 libros calificados
rating_users2=pd.read_sql(''' select "userId" as user_id,
                         count(*) as cnt_rat
                         from ratings
                         group by "userId"
                         having cnt_rat <=300
                         order by cnt_rat asc
                         ''',conn )

In [ ]:
##Graficamos para observar la distribuciòn con la restricciòn
fig  = px.histogram(rating_users2, x= 'cnt_rat', title= 'Hist frecuencia de numero de calificaciones por usario')
fig.show()


In [ ]:
rating_users2.describe()

En este caso vemos una mejor distribucion y la desviacion disminuyò mas de 200 puntos, vemos que hay valores mas cercanas y en ese caso hay 527 datos, es decir, no hubo una reducciòn significativa de datos

In [ ]:
#### verificar cuantas calificaciones tiene cada pelicula
rating_movies=pd.read_sql(''' select movieId ,
                         count(*) as cnt_rat
                         from ratings
                         group by "movieId"
                         order by cnt_rat desc
                         ''',conn )
rating_movies

In [ ]:
### analizar distribucion de calificaciones por pelicula
rating_movies.describe()

En la distribuciòn podemos ver que la pelicula con mas calificaciones es 329 y la mas  baja 1, es decir hay una gran variabiliad, el 75% de las pelicula tiene menos de 9 calificaciones, es decir, generalmente son pocas las calificaciones que recib.

In [ ]:
####Filtrar peliculas que tengan mas de 50 calificiones
rating_movies2=pd.read_sql(''' select movieId as movie,
                         count(*) as cnt_rat
                         from ratings
                         group by "movieId"
                         having cnt_rat>=50
                         order by cnt_rat desc
                         ''',conn )
rating_movies2

In [ ]:
##Anaalizamos la distribucion
rating_movies2.describe()

In [ ]:
fig  = px.histogram(rating_movies2, x= 'cnt_rat', title= 'Hist frecuencia de numero de calificaciones para cada pelicula')
fig.show()


In [ ]:
##¿Cuál es la película con el rating promedio más bajo que tiene al menos 100 calificaciones?
avr_min = pd.read_sql("""
    SELECT m.title, AVG(r.rating) AS promedio_calificacion
    FROM movies m
    JOIN ratings r ON m.movieId = r.movieId
    GROUP BY m.title
    HAVING COUNT(r.rating) >= 100
    ORDER BY promedio_calificacion
    LIMIT 1""",conn)
avr_min

In [ ]:
#### ¿Cuál es la película con la calificación promedio más alta?
consulta_sql = """
    SELECT m.title, AVG(r.rating) as calificacion
    FROM movies m
    LEFT JOIN ratings r ON m.movieId = r.movieId
    GROUP BY m.title
    ORDER BY calificacion DESC
    LIMIT 1
"""
pd.read_sql(consulta_sql, conn)

In [ ]:
#### ¿Cuántos usuarios han calificado al menos 50 películas diferentes?
consulta_sql = """
    SELECT m.title, AVG(r.rating) as calificacion
    FROM movies m
    LEFT JOIN ratings r ON m.movieId = r.movieId
    GROUP BY m.title
    ORDER BY calificacion DESC
    LIMIT 1
"""
pd.read_sql(consulta_sql, conn)

In [ ]:
 fn.ejecutar_sql('/content/drive/MyDrive/preprocesamientoSQL.sql', curr)

In [ ]:
curr.execute("select name from sqlite_master where type='table' ")
curr.fetchall()






In [ ]:
pd.read_sql('select count(*) from movies', conn)
pd.read_sql('select count(*) from ratings', conn)
pd.read_sql('select count(*) from usuarios_sel', conn)
pd.read_sql('select count(*) from movies_sel', conn)
pd.read_sql('select count(*) from ratings_final', conn)
pd.read_sql('select count(*) from full_ratings', conn)

In [ ]:

ratings=pd.read_sql('select * from full_ratings',conn)
print(ratings.duplicated().sum()) # al cruzar tablas a veces se duplican registros
print(ratings.info())
print(ratings.head(10))

In [ ]:

movies=pd.read_sql("""select * from movies""", conn)
genres =movies['genres'].str.split('|')
te = TransactionEncoder()
genres = te.fit_transform(genres)
genres = pd.DataFrame(genres, columns = te.columns_)